In [67]:
'NamedElement.namespace' in accessors

False

In [57]:
from util import parse_xmi, DotDict, get_generalization, camel_to_snake, snake_to_camel
from os import path, remove
from textwrap import wrap, indent
from networkx import DiGraph, topological_sort, simple_cycles
from re import compile as re_compile

BASE_DIR = filename = '../django_xmi/models/'
FIELD_MAPPINGS = dict(boolean=dict(name='BooleanField', attrs=[]),
                      string=dict(name='CharField', attrs=['max_length=255']),
                      real=dict(name='FloatField', attrs=[]),
                      integer=dict(name='IntegerField', attrs=[]),
                      # TODO: figure out better field for this
                      unlimited_natural=dict(name='IntegerField', attrs=[]))

RESERVED_TERMS = __builtin__.__dir__() + ['False', 'class', 'finally', 'is', 'return', 'None', 'continue',
                                          'for', 'lambda', 'try', 'True', 'def', 'from', 'nonlocal', 'while',
                                          'and', 'del', 'global', 'not', 'with', 'as', 'elif', 'if', 'or',
                                          'yield', 'assert', 'else', 'import', 'pass', 'break', 'except',
                                          'in', 'raise']
RESERVED_TERMS = tuple(term for term in RESERVED_TERMS if '_' != term[0])
INDENT = ' ' * 4
TROUBLE_ACCESSORS = {'directed_relationship_property_path.source_context',
                     'directed_relationship_property_path.source_property_path',
                     'directed_relationship_property_path.target_context',
                     'directed_relationship_property_path.target_property_path',
                     'element_group.member',
                     'element_group.ordered_memeber',
                     'named_element.namespace',
                     'namespace.named_element',
                     'participant_property.base_property',
                     'participant_property.end',
                     'requirement.derived',
                     'requirement.derived_from',
                     'requirement.master',
                     'requirement.refined_by',
                     'requirement.satisfied_by',
                     'requirement.traced_to',
                     'requirement.verified_by',
                     'value_type.quantity_kind',
                     'value_type.unit'}

# Load XMI from OMG

In [2]:
uml = parse_xmi('http://www.omg.org/spec/UML/20131001/UML.xmi')

In [3]:
sysml = parse_xmi('http://www.omg.org/spec/SysML/20150709/SysML.xmi')

# Parse elements into a `DotDict`

In [4]:
elements = DotDict({})
profiles = {'UML': uml.Package.packagedElement,
            'SysML': sysml.Profile.packagedElement}
ignore = re_compile('^[aeAE]_')
ascii_fix = re_compile(r'[^\x00-\x7F]+')

for profile_name, profile in profiles.items():
    for package_name, package in profile.items():
        if 'deprecated' in package_name.lower():
            continue
        for elem_name, element in package.packagedElement.items():
            element.update({'__package__': package_name,
                            '__profile__': profile_name,
                            '__ignore__': bool(ignore.match(elem_name)),
                            '__modelclass__': get_generalization(element) or 'models.Model',
                            '__docstring__': element.get('ownedComment', {}).get('body', ''),
                            '__is_abstract__': element.get('isAbstract', False)})
            element.__docstring__ = ascii_fix.sub("'", element.__docstring__)
            for attr_name in element.get('ownedAttribute', {}).keys():
                attr = element['ownedAttribute'][attr_name]
                if 'ownedComment' in attr:
                    attr['help_text'] = ascii_fix.sub("'", attr['ownedComment'].get('body', ''))
            elements[camel_to_snake(element.name)] = element

In [5]:
dependency_graph = DiGraph()
BASE_TYPES = ('element',)

for elem_name, element in elements.items():
    if element.__ignore__:
        continue

    if elem_name in BASE_TYPES:
        element.__modelclass__ = 'models.Model'

    if ',' in element.__modelclass__:
        for dependency in element.__modelclass__.split(', '):
            dependency_graph.add_edge(elem_name, camel_to_snake(dependency))
    else:
        dependency_graph.add_edge(elem_name, camel_to_snake(element.__modelclass__))

sorted_elements = list(topological_sort(dependency_graph))[:-1]
sorted_elements.reverse()

# Declare Helper Functions

In [6]:
def make_name_safe(name):
    name = camel_to_snake(name)
    if name in RESERVED_TERMS:
        # PEP-8's recommends adding an '_' after the name, but that is not allowed in Django
        name = 'has_' + name
    name = name.replace('__', '_')
    return name

In [7]:
def write_class(element, file):
    to_subclass = []
    to_mapping = []
    classes = tuple(kls.strip() for kls in element.__modelclass__.split(','))
    if classes == ('models.Model',):
        to_subclass = classes
    else:
        for class_ in classes:
            # TODO: testing something, remove False, but it is working better this way, may have to remove abstract models...
            if False and elements[camel_to_snake(class_)].get('isAbstract', False):
                to_subclass += [class_]
            else:
                to_mapping += [class_]

    file.write('class {}({}):\n'.format(element.name, ', '.join(to_subclass or ['models.Model'])))
    return to_mapping

In [8]:
def write_literals(literals, file, element_name):
    choices = []
    i = -1
    use_descriptions = all('ownedComment' in lit for lit in literals.values())
    for term, literal in literals.items():
        i += 1
        value = literal.ownedComment.body if use_descriptions else term
        code = "'{}'".format(term.lower()) if use_descriptions else i
        
        file.write(INDENT + '{constant} = {code}\n'.format(constant=term.upper(), code=code))
        choice = "({code}, '{value}'),\n".format(code=term.upper(), value=value)
        
        if len(choice) > 80:
            ch_ind = ' ' * (10 + len(choice.split(',')[0])) 
            joint = " ' +\n" + ch_ind + "'"
            choice = joint.join(wrap(choice, 80)) + '\n'
        choices.append(choice)
    
    if i >= 0:
        field = 'CharField(max_length=255, ' if use_descriptions else 'IntegerField('
        file.write(INDENT + 'CHOICES = (\n')
        file.write(INDENT*2 + (INDENT*2).join(choices))
        file.write(INDENT + ')\n\n')

        field_str = INDENT + '{element_name} = models.{field}choices=CHOICES, default={default})\n'
        file.write(field_str.format(element_name=element_name,
                                    field=field,
                                    default=term.upper()))
        return True
    else:
        return False

In [62]:
def write_attributes(attributes, file, element_name):
    """Write the ownedAttributes to the Django models file."""
    
    # TODO: this should probably be somewhere else
    if all(subattr in attributes for subattr in ('id', 'name', 'type')):
        element['ownedAttribute'] = DotDict({})
        element['ownedAttribute'][attributes.name] = DotDict(attributes)
        attributes = element.get('ownedAttribute', {})
        
    found_attributes = False
    redefined = {}
    
    """def __init__(self, *args, **kwargs):
    if 'value' not in kwargs:
        kwargs['value'] = 9
    super(Bar, self).__init__(*args, **kwargs)
    """
    for attr_name in sorted(attributes.keys()):
        
        attr = attributes[attr_name]
        attr_name = make_name_safe(attr_name)
        attr.name = attr_name
        
        # If the model is redefining an attribute in a superclass,
        # we need to handle it differently due to Django's limitations
        if all(prop in attr for prop in ('redefinedProperty', 'defaultValue')):
            redefined[attr.name] = attr
            continue
            
        # Check accessor
        accessor = element_name + '.' + attr_name
        reverse_accessor = snake_to_camel(attr_name) + '.' + element_name
        if reverse_accessor in accessors:
            continue
        else:
            accessors.append(reverse_accessor)
            
        added_plus = '+' if accessor in TROUBLE_ACCESSORS else ''

        subattrs = []

        if isinstance(attr.type, str):
            attr.__field__ = 'ForeignKey'
            attr.__other__ = attr.type
            subattrs += ["related_name='%(app_label)s_%(class)s_{}{}'".format(attr_name, added_plus)]
        elif isinstance(attr.type, dict):
            if 'idref' in attr.type:
                attr.__field__ = 'ForeignKey'
                attr.__other__ = attr.type['idref'].split('_')[-1]
                subattrs += ["related_name='%(app_label)s_%(class)s_{}{}'".format(attr_name, added_plus)]
            elif 'href' in attr.type:
                href = camel_to_snake(attr.type['href'].split('#')[-1])
                if href in FIELD_MAPPINGS:
                    attr.__field__ = FIELD_MAPPINGS[href]['name']
                    subattrs += FIELD_MAPPINGS[href]['attrs']
                else:
                    attr.__field__ = 'ForeignKey'
                    attr.__other__ = attr.type['href'].split('#')[-1]
                    subattrs += ["related_name='%(app_label)s_%(class)s_{}{}'".format(attr_name, added_plus)]

        field_str = '    {name} = models.{__field__}'.format(**attr)
        
        if attr.get('__other__', None):
            if attr.__other__ == element.name:
                attr.__other__ = 'self'
            subattrs = ["'{}'".format(attr.__other__)] + subattrs

        help_text = attr.get('help_text', '')
        if help_text:
            help_text = help_text.replace("'", '"')
            help_str = "help_text='{}'".format(help_text)
            
            # If the field declaration will be too long, wrap the help text
            field_len = len(field_str)
            if field_len + len(', '.join(subattrs)) + len(help_str) > 112:
                help_ind = ' ' * (len(field_str) + 1)
                joint = " ' +\n" + help_ind + "'"
                prepend = ('\n' + help_ind) if subattrs else ''
                help_str = prepend + joint.join(wrap(help_str, 112 - field_len))
            
            subattrs += [help_str]
        
        file.write(field_str + '({})\n'.format(', '.join(subattrs)))
        found_attributes = True
        
    if redefined:
        to_write = []
        
        for name, redefine in redefined.items():
            if 'defaultValue' in redefine and 'instance' in redefine.defaultValue:
                default = redefine.defaultValue.instance.split('-')[-1]
                to_write += [INDENT * 2 + "if '{}' not in kwargs:\n".format(name)]
                to_write += [INDENT * 3 + "kwargs['{}'] = '{}'\n".format(name, default)]
            else:
                print("Could not redefine {}.{}!".format(element_name, name))
        if to_write:
            file.write('\n' + INDENT + 'def __init__(self, *args, **kwargs):\n')
            for line in to_write:
                file.write(line)
            file.write(INDENT * 2 + 'super({name}).__init__(*args, **kwargs)\n'.format(**element))

    return found_attributes

In [60]:
def write_operations(operations, file, element):
    if not operations:
        return False
    
    # TODO: this should probably be somewhere else
    if all(subattr in operations for subattr in ('id', 'name', 'type')):
        element['ownedOperation'] = DotDict({})
        element['ownedOperation'][operations.name] = DotDict(operations)
        operations = element.get('ownedOperation', {})
    
    for op in operations.values():
        file.write('\n')
        
        name = op.name
        if name in element.get('ownedAttribute', {}):
            name += '_operation'
        comment = op.get('ownedComment', {}).get('body', '')
        ocl = op.get('ownedRule', {}).get('specification', {}).get('body', '')
            
        file.write(INDENT + 'def {}(self):\n'.format(camel_to_snake(name)))
        if comment:
            file.write(INDENT * 2 + '"""\n')
            for line in [indent(s, ' ' * 4) for s in wrap('{}\n'.format(comment), 104)]:
                file.write(INDENT + line + '\n')
            if not ocl:
                file.write(INDENT * 2 + '"""\n')
        if ocl:
            ocl = INDENT * 3 + ocl.replace('\n\n', '\n').replace('\n', '\n' + INDENT * 3)
            if comment:
                file.write('\n')
            else:
                file.write(INDENT * 2 + '"""\n')
            file.write(INDENT * 2 + '.. ocl::\n')
            file.write('{}\n'.format(ocl))
            file.write(INDENT * 2 + '"""\n')
        file.write(INDENT * 2 + 'pass\n')
        return True

In [51]:
def write_meta(element, file):
    found_meta = False
    # TODO: set this to false to test something, which seems to be working, so we may have to do away with abstract models
    #       if so, we need to duplicate the methods in the subclasses manually...  ugh...
    if False and element.__is_abstract__:
        file.write('\n' + INDENT + 'class Meta:\n' + INDENT * 2 + 'abstract = True\n')
        found_meta = True
    return found_meta

# Write Django models to files

In [52]:
%pdb on

Automatic pdb calling has been turned ON


In [63]:
loaded = []
accessors = []

for profile in ('uml', 'sysml'):
    filename = path.join(BASE_DIR, "{}.py".format(profile))
    if path.exists(filename):
        remove(filename)
    with open(filename, 'w') as file:
        file.write('from django.db import models\n')
        for other in loaded:
            file.write('from .{} import *\n'.format(other))
    loaded.append(profile)
        
for element_name in sorted_elements:
    if element_name not in elements:
        print('Could not find "{}"'.format(element_name))
        continue
    element = elements[element_name]
    
    filename = path.join(BASE_DIR, "{}.py".format(element.__profile__))
    
    with open(filename, 'a') as file:
        file.write('\n\n')
        
        to_mapping = write_class(element, file)
        if element.__docstring__:
            file.write(INDENT + '"""\n')
            for line in [indent(s, ' ' * 4) for s in wrap('{__docstring__}\n'.format(**element), 108)]:
                file.write(line + '\n')
            file.write(INDENT + '"""\n')
        file.write('\n')
        
        file.write(INDENT + "__package__ = '{}'\n\n".format('.'.join([element.__profile__, element.__package__])))
        
        first = True
        for other in to_mapping:
            primary = ', on_delete=models.CASCADE, primary_key=True' if first else ''
            field_str = INDENT + "{} = models.OneToOneField('{}'{})\n"
            file.write(field_str.format(make_name_safe(other), other, primary))
            first = False
        write_attributes(element.get('ownedAttribute', {}), file, element_name)
        write_literals(element.get('ownedLiteral', {}), file, element_name)
        write_meta(element, file)
        write_operations(element.get('ownedOperation', {}), file, element)

Could not redefine class.is_abstract!


# Sandbox

In [32]:
elements.constraint.ownedAttribute.specification

{'__field__': 'ForeignKey',
 '__other__': 'ValueSpecification',
 'aggregation': 'composite',
 'association': 'A_specification_owningConstraint',
 'help_text': 'A condition that must be true when evaluated in order for the Constraint to be satisfied.',
 'id': 'Constraint-specification',
 'name': 'specification',
 'ownedComment': {'annotatedElement': 'Constraint-specification',
  'body': 'A condition that must be true when evaluated in order for the Constraint to be satisfied.',
  'id': 'Constraint-specification-_ownedComment.0',
  'type': 'uml:Comment'},
 'subsettedProperty': 'Element-ownedElement',
 'type': 'ValueSpecification'}

In [31]:
elements.interval_constraint.ownedAttribute.specification

{'aggregation': 'composite',
 'association': 'A_specification_intervalConstraint',
 'id': 'IntervalConstraint-specification',
 'name': 'specification',
 'ownedComment': {'annotatedElement': 'IntervalConstraint-specification',
  'body': 'The Interval that specifies the condition of the IntervalConstraint.',
  'id': 'IntervalConstraint-specification-_ownedComment.0',
  'type': 'uml:Comment'},
 'redefinedProperty': 'Constraint-specification',
 'type': 'Interval'}

In [30]:
elements.time_constraint.ownedAttribute.specification

{'aggregation': 'composite',
 'association': 'A_specification_timeConstraint',
 'help_text': 'TheTimeInterval constraining the duration.',
 'id': 'TimeConstraint-specification',
 'name': 'specification',
 'ownedComment': {'annotatedElement': 'TimeConstraint-specification',
  'body': 'TheTimeInterval constraining the duration.',
  'id': 'TimeConstraint-specification-_ownedComment.0',
  'type': 'uml:Comment'},
 'redefinedProperty': 'IntervalConstraint-specification',
 'type': 'TimeInterval'}

In [14]:
elements.named_element.ownedAttribute.visibility

{'__field__': 'ForeignKey',
 '__other__': 'VisibilityKind',
 'help_text': 'Determines whether and how the NamedElement is visible outside its owning Namespace.',
 'id': 'NamedElement-visibility',
 'lowerValue': {'id': 'NamedElement-visibility-_lowerValue',
  'type': 'uml:LiteralInteger'},
 'name': 'visibility',
 'ownedComment': {'annotatedElement': 'NamedElement-visibility',
  'body': 'Determines whether and how the NamedElement is visible outside its owning Namespace.',
  'id': 'NamedElement-visibility-_ownedComment.0',
  'type': 'uml:Comment'},
 'type': 'VisibilityKind'}

In [15]:
vis = elements.packageable_element.ownedAttribute.get('visibility')

In [16]:
vis.defaultValue

{'id': 'PackageableElement-visibility-_defaultValue',
 'instance': 'VisibilityKind-public',
 'type': 'VisibilityKind'}

In [17]:
elements.visibility_kind.name

'VisibilityKind'

In [18]:
owned = set()
for elem in elements.values():
    owned = owned.union(set([key for key in elem.keys() if 'owned' in key.lower()]))
{k: k.replace('owned', '').lower() + 's' for k in owned if k[:5] == 'owned'}

{'ownedAttribute': 'attributes',
 'ownedComment': 'comments',
 'ownedEnd': 'ends',
 'ownedLiteral': 'literals',
 'ownedOperation': 'operations',
 'ownedRule': 'rules'}

In [32]:

err = """cerebral.DirectedRelationshipPropertyPath.source_context: (fields.E304) Reverse accessor for 'Di
rectedRelationshipPropertyPath.source_context' clashes with reverse accessor for 'DirectedRelati
onshipPropertyPath.target_context'.
        HINT: Add or change a related_name argument to the definition for 'DirectedRelationshipP
ropertyPath.source_context' or 'DirectedRelationshipPropertyPath.target_context'.
cerebral.DirectedRelationshipPropertyPath.source_property_path: (fields.E304) Reverse accessor f
or 'DirectedRelationshipPropertyPath.source_property_path' clashes with reverse accessor for 'Di
rectedRelationshipPropertyPath.target_property_path'.
        HINT: Add or change a related_name argument to the definition for 'DirectedRelationshipP
ropertyPath.source_property_path' or 'DirectedRelationshipPropertyPath.target_property_path'.
cerebral.DirectedRelationshipPropertyPath.target_context: (fields.E304) Reverse accessor for 'Di
rectedRelationshipPropertyPath.target_context' clashes with reverse accessor for 'DirectedRelati
onshipPropertyPath.source_context'.
        HINT: Add or change a related_name argument to the definition for 'DirectedRelationshipP
ropertyPath.target_context' or 'DirectedRelationshipPropertyPath.source_context'.
cerebral.DirectedRelationshipPropertyPath.target_property_path: (fields.E304) Reverse accessor f
or 'DirectedRelationshipPropertyPath.target_property_path' clashes with reverse accessor for 'Di
rectedRelationshipPropertyPath.source_property_path'.
        HINT: Add or change a related_name argument to the definition for 'DirectedRelationshipP
ropertyPath.target_property_path' or 'DirectedRelationshipPropertyPath.source_property_path'.
cerebral.ElementGroup.member: (fields.E304) Reverse accessor for 'ElementGroup.member' clashes w
ith reverse accessor for 'ElementGroup.ordered_memeber'.
        HINT: Add or change a related_name argument to the definition for 'ElementGroup.member'
or 'ElementGroup.ordered_memeber'.
cerebral.ElementGroup.ordered_memeber: (fields.E304) Reverse accessor for 'ElementGroup.ordered_
memeber' clashes with reverse accessor for 'ElementGroup.member'.
        HINT: Add or change a related_name argument to the definition for 'ElementGroup.ordered_
memeber' or 'ElementGroup.member'.
cerebral.Namespace.named_element: (fields.E302) Reverse accessor for 'Namespace.named_element' c
lashes with field name 'NamedElement.namespace'.
        HINT: Rename field 'NamedElement.namespace', or add/change a related_name argument to th
e definition for field 'Namespace.named_element'.
cerebral.Namespace.named_element: (fields.E303) Reverse query name for 'Namespace.named_element'
 clashes with field name 'NamedElement.namespace'.
        HINT: Rename field 'NamedElement.namespace', or add/change a related_name argument to th
e definition for field 'Namespace.named_element'.
cerebral.ParticipantProperty.base_property: (fields.E304) Reverse accessor for 'ParticipantPrope
rty.base_property' clashes with reverse accessor for 'ParticipantProperty.end'.
        HINT: Add or change a related_name argument to the definition for 'ParticipantProperty.b
ase_property' or 'ParticipantProperty.end'.
cerebral.ParticipantProperty.end: (fields.E304) Reverse accessor for 'ParticipantProperty.end' c
lashes with reverse accessor for 'ParticipantProperty.base_property'.
        HINT: Add or change a related_name argument to the definition for 'ParticipantProperty.e
nd' or 'ParticipantProperty.base_property'.
cerebral.Requirement.derived: (fields.E304) Reverse accessor for 'Requirement.derived' clashes w
ith reverse accessor for 'Requirement.derived_from'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.derived'
or 'Requirement.derived_from'.
cerebral.Requirement.derived: (fields.E304) Reverse accessor for 'Requirement.derived' clashes w
ith reverse accessor for 'Requirement.master'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.derived'
or 'Requirement.master'.
cerebral.Requirement.derived_from: (fields.E304) Reverse accessor for 'Requirement.derived_from'
 clashes with reverse accessor for 'Requirement.derived'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.derived_f
rom' or 'Requirement.derived'.
cerebral.Requirement.derived_from: (fields.E304) Reverse accessor for 'Requirement.derived_from'
 clashes with reverse accessor for 'Requirement.master'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.derived_f
rom' or 'Requirement.master'.
cerebral.Requirement.master: (fields.E304) Reverse accessor for 'Requirement.master' clashes wit
h reverse accessor for 'Requirement.derived'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.master' o
r 'Requirement.derived'.
cerebral.Requirement.master: (fields.E304) Reverse accessor for 'Requirement.master' clashes wit
h reverse accessor for 'Requirement.derived_from'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.master' o
r 'Requirement.derived_from'.
cerebral.Requirement.refined_by: (fields.E304) Reverse accessor for 'Requirement.refined_by' cla
shes with reverse accessor for 'Requirement.satisfied_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.refined_b
y' or 'Requirement.satisfied_by'.
cerebral.Requirement.refined_by: (fields.E304) Reverse accessor for 'Requirement.refined_by' cla
shes with reverse accessor for 'Requirement.traced_to'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.refined_b
y' or 'Requirement.traced_to'.
cerebral.Requirement.refined_by: (fields.E304) Reverse accessor for 'Requirement.refined_by' cla
shes with reverse accessor for 'Requirement.verified_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.refined_b
y' or 'Requirement.verified_by'.
cerebral.Requirement.satisfied_by: (fields.E304) Reverse accessor for 'Requirement.satisfied_by'
 clashes with reverse accessor for 'Requirement.refined_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.satisfied
_by' or 'Requirement.refined_by'.
cerebral.Requirement.satisfied_by: (fields.E304) Reverse accessor for 'Requirement.satisfied_by'
 clashes with reverse accessor for 'Requirement.traced_to'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.satisfied
_by' or 'Requirement.traced_to'.
cerebral.Requirement.satisfied_by: (fields.E304) Reverse accessor for 'Requirement.satisfied_by'
 clashes with reverse accessor for 'Requirement.verified_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.satisfied
_by' or 'Requirement.verified_by'.
cerebral.Requirement.traced_to: (fields.E304) Reverse accessor for 'Requirement.traced_to' clash
es with reverse accessor for 'Requirement.refined_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.traced_to
' or 'Requirement.refined_by'.
cerebral.Requirement.traced_to: (fields.E304) Reverse accessor for 'Requirement.traced_to' clash
es with reverse accessor for 'Requirement.satisfied_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.traced_to
' or 'Requirement.satisfied_by'.
cerebral.Requirement.traced_to: (fields.E304) Reverse accessor for 'Requirement.traced_to' clash
es with reverse accessor for 'Requirement.verified_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.traced_to
' or 'Requirement.verified_by'.
cerebral.Requirement.verified_by: (fields.E304) Reverse accessor for 'Requirement.verified_by' c
lashes with reverse accessor for 'Requirement.refined_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.verified_
by' or 'Requirement.refined_by'.
cerebral.Requirement.verified_by: (fields.E304) Reverse accessor for 'Requirement.verified_by' c
lashes with reverse accessor for 'Requirement.satisfied_by'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.verified_
by' or 'Requirement.satisfied_by'.
cerebral.Requirement.verified_by: (fields.E304) Reverse accessor for 'Requirement.verified_by' c
lashes with reverse accessor for 'Requirement.traced_to'.
        HINT: Add or change a related_name argument to the definition for 'Requirement.verified_
by' or 'Requirement.traced_to'.
cerebral.ValueType.quantity_kind: (fields.E304) Reverse accessor for 'ValueType.quantity_kind' c
lashes with reverse accessor for 'ValueType.unit'.
        HINT: Add or change a related_name argument to the definition for 'ValueType.quantity_ki
nd' or 'ValueType.unit'.
cerebral.ValueType.unit: (fields.E304) Reverse accessor for 'ValueType.unit' clashes with revers
e accessor for 'ValueType.quantity_kind'.
        HINT: Add or change a related_name argument to the definition for 'ValueType.unit' or 'V
alueType.quantity_kind'.
"""

err_regex = re_compile(r"'[a-zA-Z_]*.[a-zA-Z_]*'")
set(err_acc[1:-1] for err_acc in err_regex.findall(err))


{'DirectedRelationshipPropertyPath.source_context',
 'DirectedRelationshipPropertyPath.source_property_path',
 'DirectedRelationshipPropertyPath.target_context',
 'DirectedRelationshipPropertyPath.target_property_path',
 'ElementGroup.member',
 'ElementGroup.ordered_memeber',
 'NamedElement.namespace',
 'Namespace.named_element',
 'ParticipantProperty.base_property',
 'ParticipantProperty.end',
 'Requirement.derived',
 'Requirement.derived_from',
 'Requirement.master',
 'Requirement.refined_by',
 'Requirement.satisfied_by',
 'Requirement.traced_to',
 'Requirement.verified_by',
 'ValueType.quantity_kind',
 'ValueType.unit'}

In [19]:
elements.element.ownedOperation.allOwnedElements.ownedParameter.lowerValue

{'id': 'Element-allOwnedElements-result-_lowerValue',
 'type': 'uml:LiteralInteger'}

In [20]:
elements.visibility_kind.ownedLiteral.package.ownedComment.body

'A NamedElement with package visibility is visible to all Elements within the nearest enclosing Package (given that other owning Elements have proper visibility). Outside the nearest enclosing Package, a NamedElement marked as having package visibility is not visible.  Only NamedElements that are not owned by Packages can be marked as having package visibility.'

In [21]:
keys = set()
for elem in elements.values():
    keys = keys.union(set(elem.keys()))
keys

{'__docstring__',
 '__ignore__',
 '__is_abstract__',
 '__modelclass__',
 '__package__',
 '__profile__',
 'generalization',
 'id',
 'isAbstract',
 'isDerived',
 'memberEnd',
 'name',
 'navigableOwnedEnd',
 'ownedAttribute',
 'ownedComment',
 'ownedEnd',
 'ownedLiteral',
 'ownedOperation',
 'ownedRule',
 'packagedElement',
 'type',
 'uuid'}

# TODOs

### Parse rules
* This is a maybe
* All these elements have rules:

In [ ]:
sorted([k for k, v in elements.items() if 'ownedRule' in v])

### Remove the reverse accessors which Django creates automatically
* e.g., `Namespace.named_element: (fields.E302) Reverse accessor for 'Namespace.named_element' clashes with field name 'NamedElement.namespace'.`

### Parse multiplicity relationships
* Look at how OSLC's SysML profile figure out how to determine if a relationship is `zero-to-one`, `one-to-many`, or `zero-to-many`

### Separate loading UML from the profiles
* right now things just "get loaded", make it so UML is loaded as a package, or a profile loads its imported packages

### Fix issue with fields referencing abstract models
* May have to do away with declaring things as `abstract` and make everything inherit from `models.Model` and use `OneToOneField`